**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

The function act returns an action given a state. It distinguishes the training situation during which it makes a random exploration choice of action with probability ```epsilon``` and chooses the ideal action the rest of the time according to what it has learned up till now. When the context does not correspond to training, it does not make any random exploration and simply chooses the action it thinks best.

```epsilon``` is essential because it obliges the agent to take random actions regularly so as to explore all the action space and not stick to best choices learned on the basis of few experiments.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1  # self.position[:, -2:] = -1 ## ????
        self.position[:, -2:] = -1 ## ????

        
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=25#?? # set small when debugging
epochs_test=15#?? # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` represents the state of the environment, it keeps track of where there is cheese and where there is poison and is updated accordingly to the agent's actions.

The array ```position``` indicates the position of the rat on the grid (which is padded with 2 cells with value -1). It is used to generate the state by cropping a $5\times 5$ window around the current position of the rat.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #pass
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix='', keep_exploring = False):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=keep_exploring)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            else:#if reward < 0:
                lose -= reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.0/13.0. Average score (-6.0)
Win/lose count 9.5/7.0. Average score (-1.75)
Win/lose count 12.5/14.0. Average score (-1.6666666666666667)
Win/lose count 7.5/15.0. Average score (-3.125)
Win/lose count 12.5/20.0. Average score (-4.0)
Win/lose count 14.0/13.0. Average score (-3.1666666666666665)
Win/lose count 12.0/15.0. Average score (-3.142857142857143)
Win/lose count 12.0/10.0. Average score (-2.5)
Win/lose count 8.5/16.0. Average score (-3.0555555555555554)
Win/lose count 12.5/22.0. Average score (-3.7)
Win/lose count 7.0/6.0. Average score (-3.272727272727273)
Win/lose count 11.0/20.0. Average score (-3.75)
Win/lose count 9.5/13.0. Average score (-3.730769230769231)
Win/lose count 13.0/18.0. Average score (-3.8214285714285716)
Win/lose count 6.5/13.0. Average score (-4.0)
Final score: -4.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




_______________________________________________________
We have :

$\DeclareMathOperator*{\argmax}{arg\,max}$
\begin{align*}
Q^{\pi}(s,a) &= \mathbb{E}_{p^{\pi}}[\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) \mid s_0 =s, a_0=a] \\
    &= \mathbb{E}_{p^{\pi}}[r(s, a) + \sum_{t=1}^{\infty} \gamma^t r(s_t, a_t) \mid s_0 =s, a_0=a] \\
    &= \mathbb{E}_{p^{\pi}}[r(s, a) + \gamma\sum_{t=0}^{\infty} \gamma^t r(s_{t+1}, a_{t+1}) \mid s_0 =s, a_0=a] \\
    &= \mathbb{E}_{s' \sim p(\cdot \mid s, a), \:\: a' \sim \pi}[r(s, a) + \gamma Q^{\pi}(s',a')] \\
\end{align*}

and from there :
\begin{align*}
    \max_{\pi} Q^{\pi}(s, a) &= \max_{\pi} \mathbb{E}_{s' \sim p(\cdot \mid s, a), \:\: a' \sim \pi}[r(s, a) + \gamma Q^{\pi}(s',a')] \\
    &= \mathbb{E}_{s' \sim p(\cdot \mid s, a), \:\: a' \sim \pi^*}[r(s, a) + \gamma Q^{*}(s',a')] \\
    &= \mathbb{E}_{s' \sim p(\cdot \mid s, a)}[r(s, a) + \gamma \max_{a'}Q^{*}(s',a')] \\
\end{align*}

because $a' \sim \pi^*$ corresponds to $a' = \argmax_a Q^*(s, a)$


When the $Q$ function is parametrized by $\theta$, a reasonable (bootstrap) estimate of $Q^*(s_t, a_t)$ is $r(s_t, a_t) + \gamma \max_a Q(s_{t+1}, a)$ according to the definition of $Q^*$, so that the squared difference between the two is a good objective which corresponds to
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r + \gamma \max_{a'} Q(s',a',\theta) - Q(s,a,\theta)\Vert^{2}.
\end{equation*}

but we will only optimize this stochastically because we only observe one realization of $s' \sim p(.|s,a)$ at a time


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[-self.max_memory:]
        #pass

    def random_access(self):
        le = len(self.memory)
        if le > 0:
            return self.memory[np.random.randint(le)]
        #else:
        #    pass

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            st, n_st, ac, rew, game_o = self.memory.random_access()
            input_states[i,:,:,:] = st
            target_q[i] = self.model.predict(np.array([st]))
            
            if game_o:#ver_:
                ######## FILL IN
                target_q[i, ac] = rew
            else:
                ######## FILL IN
                target_q[i, ac] = rew + self.discount*np.max(self.model.predict(np.array([n_st])))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        model.add(keras.layers.Flatten(input_shape=(5,5,2)))
        state_shape  = 5*5*2
        model.add(Dense(100, input_dim=state_shape, activation="relu"))

        model.add(Dense(100, activation="relu"))
        
        model.add(Dense(75, activation="relu"))
        model.add(Dense(25, activation="relu"))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-6, momentum=0.5), "mse")
        self.model = model
    
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])))#.reshape((1,-1))))
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/025 | Loss 0.0531 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/025 | Loss 0.0512 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/025 | Loss 0.0008 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/025 | Loss 0.0018 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/025 | Loss 0.0025 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/025 | Loss 0.0552 | Win/lose count 2.5/0 (2.5)
Epoch 006/025 | Loss 0.0015 | Win/lose count 0.5/2.0 (-1.5)
Epoch 007/025 | Loss 0.0011 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/025 | Loss 0.0473 | Win/lose count 3.0/1.0 (2.0)
Epoch 009/025 | Loss 0.0028 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/025 | Loss 0.0021 | Win/lose count 0.5/3.0 (-2.5)
Epoch 011/025 | Loss 0.0018 | Win/lose count 6.0/6.0 (0.0)
Epoch 012/025 | Loss 0.0063 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/025 | Loss 0.0058 | Win/lose count 5.0/4.0 (1.0)
Epoch 014/025 | Loss 0.0014 | Win/lose count 2.5/3.0 (-0.5)
Epoch 015/025 | Loss 0.0121 | Win/lose count 4.5/3.0 (1.5)
Epoch 016/025 | Loss 0.0022 | Win/lose co

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(20, (3, 3), padding="same", activation="relu"))
        
        model.add(Conv2D(20, (3, 3), padding="same", activation="relu"))
        
        model.add(keras.layers.Flatten())

        model.add(Dense(25, activation="relu"))
        
        model.add(Dense(4))#, activation="relu"))

        model.compile(sgd(lr=lr, decay=1e-6, momentum=0.5), "mse")
        self.model = model
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])))#.reshape((1,-1))))


In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/025 | Loss 0.0124 | Win/lose count 7.5/3.0 (4.5)
Epoch 001/025 | Loss 0.0126 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/025 | Loss 0.0022 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/025 | Loss 0.0040 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/025 | Loss 0.0031 | Win/lose count 1.0/4.0 (-3.0)
Epoch 005/025 | Loss 0.0049 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/025 | Loss 0.0079 | Win/lose count 4.0/0 (4.0)
Epoch 007/025 | Loss 0.0072 | Win/lose count 2.0/4.0 (-2.0)
Epoch 008/025 | Loss 0.0033 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/025 | Loss 0.0062 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/025 | Loss 0.0099 | Win/lose count 1.5/2.0 (-0.5)
Epoch 011/025 | Loss 0.0156 | Win/lose count 4.0/9.0 (-5.0)
Epoch 012/025 | Loss 0.0118 | Win/lose count 4.5/5.0 (-0.5)
Epoch 013/025 | Loss 0.0131 | Win/lose count 1.5/3.0 (-1.5)
Epoch 014/025 | Loss 0.0050 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/025 | Loss 0.0018 | Win/lose count 2.0/0 (2.0)
Epoch 016/025 | Loss 0.0148 | Win/lose count 2.5/6.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)#0.3
agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0.0. Average score (0.0)
Win/lose count 1.0/0.0. Average score (0.5)
Win/lose count 2.0/0.0. Average score (1.0)
Win/lose count 2.5/0.0. Average score (1.375)
Win/lose count 2.0/0.0. Average score (1.5)
Win/lose count 1.0/0.0. Average score (1.4166666666666667)
Win/lose count 6.0/1.0. Average score (1.9285714285714286)
Win/lose count 0/0.0. Average score (1.6875)
Win/lose count 1.0/0.0. Average score (1.6111111111111112)
Win/lose count 6.0/0.0. Average score (2.05)
Win/lose count 1.0/0.0. Average score (1.9545454545454546)
Win/lose count 4.5/0.0. Average score (2.1666666666666665)
Win/lose count 2.0/0.0. Average score (2.1538461538461537)
Win/lose count 3.0/0.0. Average score (2.2142857142857144)
Win/lose count 0.5/0.0. Average score (2.1)
Final score: 2.1
Test of the FC
Win/lose count 2.0/1.0. Average score (1.0)
Win/lose count 1.0/2.0. Average score (0.0)
Win/lose count 0/1.0. Average score (-0.3333333333333333)
Win/lose count 0/0.0. Average score (-0

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

We observe that the agent tends not to explore the grid, it often keeps going back and forth in a loop

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):

    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        tm = 1
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(max(0.1, np.power(tm, -0.5)))
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            tm += 1
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position =  np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1  # self.position[:, -2:] = -1 ## ????
        self.position[:, -2:] = -1 ## ????
        
        
        
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.t = self.t + 1

        reward = reward + self.board[self.x, self.y]

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        #self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1#

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/025 | Loss 0.0105 | Win/lose count 5.5/25.20000000000004 (-19.70000000000004)
Epoch 001/025 | Loss 0.0058 | Win/lose count 6.400000000000002/23.700000000000074 (-17.30000000000007)
Epoch 002/025 | Loss 0.0080 | Win/lose count 6.600000000000001/17.59999999999998 (-10.999999999999979)
Epoch 003/025 | Loss 0.0056 | Win/lose count 6.600000000000002/22.20000000000001 (-15.600000000000009)
Epoch 004/025 | Loss 0.0057 | Win/lose count 12.400000000000006/22.100000000000055 (-9.700000000000049)
Epoch 005/025 | Loss 0.0080 | Win/lose count 9.000000000000004/21.500000000000043 (-12.500000000000039)
Epoch 006/025 | Loss 0.0070 | Win/lose count 20.0/20.500000000000004 (-0.5000000000000036)
Epoch 007/025 | Loss 0.0162 | Win/lose count 8.3/20.300000000000022 (-12.000000000000021)
Epoch 008/025 | Loss 0.0062 | Win/lose count 18.6/18.800000000000008 (-0.2000000000000064)
Epoch 009/025 | Loss 0.0080 | Win/lose count 7.200000000000002/23.300000000000058 (-16.100000000000055)
Epoch 010/025 | Los

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore', keep_exploring=True)
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 17.5/2.0. Average score (15.5)
Win/lose count 8.5/1.0. Average score (11.5)
Win/lose count 14.5/7.0. Average score (10.166666666666666)
Win/lose count 17.5/4.0. Average score (11.0)
Win/lose count 22.5/3.0. Average score (12.7)
Win/lose count 19.0/5.0. Average score (12.916666666666666)
Win/lose count 24.5/3.0. Average score (14.142857142857142)
Win/lose count 13.0/7.0. Average score (13.125)
Win/lose count 21.5/0.0. Average score (14.055555555555555)
Win/lose count 10.5/4.0. Average score (13.3)
Win/lose count 17.0/4.0. Average score (13.272727272727273)
Win/lose count 19.5/3.0. Average score (13.541666666666666)
Win/lose count 22.5/3.0. Average score (14.0)
Win/lose count 17.5/4.0. Average score (13.964285714285714)
Win/lose count 9.5/4.0. Average score (13.4)
Final score: 13.4


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

We train an imitator agent. We give it the previous expert agent and let it learn by trying to make the same decisions as the expert.

We reuse most of the previous code for simplicity although the imitator is more of a classifier than an agent by reinforcement.

In [21]:
class DQN_CNN_imitator(DQN_CNN):
    def __init__(self, *args,lr=0.1, expert=None,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        if expert is None:
            print("Warning, need an expert or it won't work")
            
        self.expert = expert
        
        model = Sequential()

        model.add(Conv2D(20, (3, 3), padding="same", activation="relu"))
        
        model.add(Conv2D(20, (3, 3), padding="same", activation="relu"))

        model.add(keras.layers.Flatten())
        
        model.add(Dense(25, activation="relu"))
        
        model.add(Dense(4, activation="softmax"))
        
        model.compile(sgd(lr=lr, decay=1e-6, momentum=0.5), "categorical_crossentropy")
        self.model = model

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])))#.reshape((1,-1))))
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, np.eye(4)[self.expert.learned_act(s_),:]])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            st, expert_act = self.memory.random_access()
            input_states[i,:,:,:] = st
            target[i] = expert_act
            
            #if game_o:#ver_:
            #    ######## FILL IN
            #    target_q[i, ac] = rew
            #else:
            #    ######## FILL IN
            #    target_q[i, ac] = rew + self.discount*np.max(self.model.predict(np.array([n_st])))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        #target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target)#.reshape((self.batch_size,-1))
        return l


In [22]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
imitator = DQN_CNN_imitator(size, lr=.01, epsilon = 0.1, expert = agent, memory_size=2000, batch_size = 32,n_state=3)
train_explore(imitator, env, epochs_train, prefix='cnn_imitator_train')
HTML(display_videos('cnn_imitator_train10.mp4'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 000/025 | Loss 0.6544 | Win/lose count 6.0/26.20000000000009 (-20.20000000000009)
Epoch 001/025 | Loss 0.4043 | Win/lose count 5.800000000000001/22.300000000000058 (-16.500000000000057)
Epoch 002/025 | Loss 0.3809 | Win/lose count 8.700000000000003/19.60000000000001 (-10.900000000000006)
Epoch 003/025 | Loss 0.2868 | Win/lose count 8.4/19.000000000000007 (-10.600000000000007)
Epoch 004/025 | Loss 0.6425 | Win/lose count 11.100000000000005/18.700000000000003 (-7.599999999999998)
Epoch 005/025 | Loss 0.4670 | Win/lose count 7.400000000000002/18.9 (-11.499999999999996)
Epoch 006/025 | Loss 0.2734 | Win/lose count 14.100000000000005/18.700000000000003 (-4.599999999999998)
Epoch 007/025 | Loss 0.4368 | Win/lose count 10.500000000000004/22.20000000000006 (-11.700000000000056)
Epoch 008/025 | Loss 0.1774 | Win/lose count 19.099999999999998/17.199999999999978 (1.90000000000002)
Epoch 009/025 | 

In [23]:
test(imitator,env,epochs_test,prefix='cnn_imitator_test', keep_exploring=True)
HTML(display_videos('cnn_imitator_test10.mp4'))

Win/lose count 18.0/1.0. Average score (17.0)
Win/lose count 22.5/1.0. Average score (19.25)
Win/lose count 14.0/3.0. Average score (16.5)
Win/lose count 14.0/3.0. Average score (15.125)
Win/lose count 14.0/5.0. Average score (13.9)
Win/lose count 19.5/4.0. Average score (14.166666666666666)
Win/lose count 19.0/1.0. Average score (14.714285714285714)
Win/lose count 17.5/6.0. Average score (14.3125)
Win/lose count 17.5/3.0. Average score (14.333333333333334)
Win/lose count 12.5/1.0. Average score (14.05)
Win/lose count 15.5/5.0. Average score (13.727272727272727)
Win/lose count 4.5/0.0. Average score (12.958333333333334)
Win/lose count 11.0/0.0. Average score (12.807692307692308)
Win/lose count 18.0/3.0. Average score (12.964285714285714)
Win/lose count 26.5/5.0. Average score (13.533333333333333)
Final score: 13.533333333333333
